<a href="https://www.kaggle.com/code/mnnazza/stepcom?scriptVersionId=283216320" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

In [1]:
import os, time, random, json, traceback
from datetime import datetime, timedelta

import google.generativeai as genai
from kaggle_secrets import UserSecretsClient

# get secret
user_secrets = UserSecretsClient()
secret_value = user_secrets.get_secret("GOOGLE_API_KEY")
if not secret_value:
    raise ValueError("Secret 'GOOGLE_API_KEY' not found in Kaggle Secrets. Add it and re-run this cell.")

os.environ["GOOGLE_API_KEY"] = secret_value
try:
    genai.configure(api_key=secret_value)
except Exception:
    pass

print("✔️ Gemini configured. google-generativeai version:", getattr(genai, "__version__", "unknown"))

✔️ Gemini configured. google-generativeai version: 0.8.5


**Cell 1 - Environment Setup & Gemini SDK Configuration**

This cell initializes everything required for Stepcom to run inside Kaggle.

What this cell does

Imports core Python libraries (os, json, datetime, etc.).

Retrieves GOOGLE_API_KEY securely via Kaggle Secrets (no hard-coded keys).

Sets up the environment variable and configures the Gemini SDK.

Verifies installation by printing the SDK version.

Why this matters

Every other component - including model detection, agents, memory, and UI - depend on the Gemini client being initialized here.
If this cell fails, nothing else will work.

In [2]:
USE_HTTP_RETRY = False
retry_config = None
try:
    from google.generativeai import types
    if hasattr(types, "HttpRetryOptions"):
        retry_config = types.HttpRetryOptions(
            attempts=5, exp_base=2, initial_delay=1.0,
            http_status_codes=[429,500,503,504]
        )
        USE_HTTP_RETRY = True
        print("✔️ Using official HttpRetryOptions for retries.")
    else:
        print("ℹ️ types present but HttpRetryOptions not available. Using fallback retries.")
except Exception:
    print("ℹ️ google.generativeai.types not available. Using fallback retries.")

# fallback retry helper
def send_with_retries(chat_obj, prompt, max_attempts=5, initial_delay=1.0, backoff=2.0, jitter=0.2):
    attempt = 0
    delay = initial_delay
    last_exc = None
    while attempt < max_attempts:
        try:
            return chat_obj.send_message(prompt)
        except Exception as e:
            last_exc = e
            attempt += 1
            if attempt >= max_attempts:
                raise
            sleep_time = max(0.1, delay + random.uniform(-jitter*delay, jitter*delay))
            print(f"[fallback-retry] attempt {attempt}/{max_attempts} failed: {type(e).__name__}: {e}")
            print(f"  → retrying in {sleep_time:.2f}s...")
            time.sleep(sleep_time)
            delay *= backoff
    if last_exc:
        raise last_exc

print("Retry helper ready. USE_HTTP_RETRY=", USE_HTTP_RETRY)

ℹ️ types present but HttpRetryOptions not available. Using fallback retries.
Retry helper ready. USE_HTTP_RETRY= False


**Cell 2 - Retry Logic (Official + Fallback)**

This cell detects whether Google’s official HttpRetryOptions is available.
If not, Stepcom uses a flexible fallback retry handler.

Responsibilities

Enables stable API connections.

Prevents failures due to rate limits, slow network, or transient errors.

Provides exponential backoff, jitter, and max retry attempts.

Why it matters

Kaggle notebooks occasionally face network instability - this ensures Stepcom works smoothly even under quota or connection issues.

In [3]:
MODEL_NAME_PATH = "/kaggle/working/stepcom_model_name.txt"

print("Listing models available to this API key (may take a second)...")
try:
    models = genai.list_models()
    model_names = []
    for m in models:
        try:
            model_names.append(m.name if hasattr(m, "name") else str(m))
        except Exception:
            model_names.append(str(m))
    model_names = list(dict.fromkeys(model_names))
    for i,nm in enumerate(model_names[:50], start=1):
        print(f"{i:02d}. {nm}")
except Exception as e:
    print("Could not list models:", e)
    model_names = []

# candidate selection: prefer gemini flash/pro or gemini-flash-latest
candidates = []
for nm in model_names:
    ln = nm.lower()
    if "gemini" in ln and "flash" in ln:
        candidates.append(nm)
for nm in model_names:
    if nm not in candidates:
        candidates.append(nm)
for nm in ["models/gemini-2.5-flash","models/gemini-pro-latest","models/gemini-flash-latest"]:
    if nm not in candidates:
        candidates.append(nm)

print("\nTrying candidate models (stops on first success)")
used_model = None
for cand in candidates[:15]:
    print("Trying", cand)
    try:
        if USE_HTTP_RETRY and retry_config is not None:
            model = genai.GenerativeModel(model_name=cand, system_instruction="You are StepCom - concise and kind.", http_retry=retry_config)
        else:
            model = genai.GenerativeModel(model_name=cand, system_instruction="You are StepCom - concise and kind.")
        chat = model.start_chat(history=[])
        try:
            if USE_HTTP_RETRY:
                resp = chat.send_message("Test: say hi")
            else:
                resp = send_with_retries(chat, "Test: say hi")
            text = getattr(resp, "text", str(resp))
            print("→ success with", cand, "preview:", text[:120])
            used_model = cand
            break
        except Exception as e:
            print(" model call failed:", type(e).__name__, e)
    except Exception as e:
        print(" create model failed:", type(e).__name__, e)

if not used_model:
    used_model = "models/gemini-2.5-flash"

with open(MODEL_NAME_PATH, "w") as f:
    f.write(used_model)

MODEL_NAME = used_model
print("Selected model:", MODEL_NAME)

Listing models available to this API key (may take a second)...
01. models/embedding-gecko-001
02. models/gemini-2.5-pro-preview-03-25
03. models/gemini-2.5-flash
04. models/gemini-2.5-pro-preview-05-06
05. models/gemini-2.5-pro-preview-06-05
06. models/gemini-2.5-pro
07. models/gemini-2.0-flash-exp
08. models/gemini-2.0-flash
09. models/gemini-2.0-flash-001
10. models/gemini-2.0-flash-exp-image-generation
11. models/gemini-2.0-flash-lite-001
12. models/gemini-2.0-flash-lite
13. models/gemini-2.0-flash-lite-preview-02-05
14. models/gemini-2.0-flash-lite-preview
15. models/gemini-2.0-pro-exp
16. models/gemini-2.0-pro-exp-02-05
17. models/gemini-exp-1206
18. models/gemini-2.0-flash-thinking-exp-01-21
19. models/gemini-2.0-flash-thinking-exp
20. models/gemini-2.0-flash-thinking-exp-1219
21. models/gemini-2.5-flash-preview-tts
22. models/gemini-2.5-pro-preview-tts
23. models/learnlm-2.0-flash-experimental
24. models/gemma-3-1b-it
25. models/gemma-3-4b-it
26. models/gemma-3-12b-it
27. model

**Cell 3 - Automatic Model Selection**

This cell scans all Gemini models available to the user’s API key and automatically selects the best-performing chat model.

What it does

Lists all models accessible via your key.

Prioritizes fast + capable Gemini Flash models.

Runs a test conversation with each candidate model.

Saves the working model to:
/kaggle/working/stepcom_model_name.txt

Why it matters

Every user's API plan gives access to different models.
This auto-selection guarantees Stepcom chooses a working model instead of failing with 404 or quota issues.

In [4]:
MEMORY_PATH = "/kaggle/working/stepcom_memory.json"
EXPLAIN_CACHE_PATH = "/kaggle/working/stepcom_explain_cache.json"

default_memory = {
    "tasks": [],
    "completed": [],
    "goals": [],
    "habits": {},
    "meta": {"mode": "soft", "excuse_count": 0, "a2a_logs": []}
}

def load_json(path, default):
    if os.path.exists(path):
        try:
            with open(path, "r") as f:
                return json.load(f)
        except Exception as e:
            print("Warning loading", path, ":", e)
            return default
    return default

def save_json(path, obj):
    with open(path, "w") as f:
        json.dump(obj, f, indent=2, default=str)

memory = load_json(MEMORY_PATH, default_memory.copy())
explain_cache = load_json(EXPLAIN_CACHE_PATH, {})
print("Memory loaded. Mode:", memory["meta"].get("mode","soft"))

Memory loaded. Mode: soft


**Cell 4 - Memory System (Persistent JSON Storage)**

This cell creates and loads Stepcom’s long-term memory files.

Files created

stepcom_memory.json

stepcom_explain_cache.json

What Stepcom remembers

Habits (good & bad)

Tasks & completions

Goals (if user mentions them)

Explanations cached (so Gemini isn’t called repeatedly)

Emotion/Plan agent logs

Why this matters

Memory enables Stepcom to behave like a consistent, caring productivity partner who knows your patterns and follows up naturally.

In [5]:
def log_a2a_call(agent, prompt, response):
    entry = {"agent": agent, "prompt": (prompt or "")[:800], "response": (response or "")[:2000], "ts": datetime.now().isoformat()}
    memory["meta"].setdefault("a2a_logs", []).append(entry)
    save_json(MEMORY_PATH, memory)

print("Observability helper ready.")

Observability helper ready.


**Cell 5 - Observability Logging**

This cell logs every internal A2A (Agent-to-Agent) call.

What gets logged

Timestamp

Agent name (EmotionAgent / PlanAgent / Controller)

Prompt sent

Response generated

Why it matters

This gives transparency in agent behaviour, helps debugging issues, and is required for Kaggle Capstone evaluation.

In [6]:
def add_task(task: str):
    if not task: return "No task provided."
    memory.setdefault("tasks", []).append({"task": task, "added": datetime.now().isoformat()})
    save_json(MEMORY_PATH, memory)
    return f"Added task: {task} ✨"

def complete_task(task: str):
    for t in list(memory.get("tasks", [])):
        if t["task"].strip().lower() == task.strip().lower():
            memory.setdefault("completed", []).append({"task": task, "completed": datetime.now().isoformat()})
            memory["tasks"].remove(t)
            save_json(MEMORY_PATH, memory)
            return f"Completed: {task} 🎉"
    return "Couldn't find that task."

def add_goal(goal: str):
    if not goal: return "No goal provided."
    memory.setdefault("goals", []).append(goal)
    save_json(MEMORY_PATH, memory)
    return f"Saved goal: {goal} 🌟"

def add_habit(name: str, kind: str):
    if not name or not kind: return "Usage: add_habit(name, kind)"
    kind = kind.lower()
    if kind not in ("good","bad"): return "Habit type must be 'good' or 'bad'."
    memory.setdefault("habits", {})[name] = {"type": kind, "streak": 0, "last_done": None, "notes": []}
    save_json(MEMORY_PATH, memory)
    return f"Habit '{name}' added as a {kind} habit."

def mark_habit_done(name: str):
    h = memory.get("habits", {}).get(name)
    if not h: return "Habit not found."
    now = datetime.now()
    last_iso = h.get("last_done")
    last = None
    if last_iso:
        try:
            last = datetime.fromisoformat(last_iso)
        except:
            last = None
    if last and (now.date() - last.date()) == timedelta(days=1):
        h["streak"] += 1
    else:
        if last and now.date() == last.date():
            pass
        else:
            h["streak"] = 1
    h["last_done"] = now.isoformat()
    h.setdefault("notes", []).append({"done": now.isoformat()})
    save_json(MEMORY_PATH, memory)
    return f"Marked '{name}' done. Streak: {h['streak']} 🌱"

def get_habits():
    return memory.get("habits", {})

print("Core tools ready.")

Core tools ready.


**Cell 6 - Core Internal Tools**

These are backend functions that StepCom can trigger autonomously when needed.

Includes

Add task

Mark task completed

Save goal

Add habit (good or bad)

Mark habit done

Retrieve habits

Why it matters

Even though the UI is chat-only, Stepcom still needs to store structured habit/task data internally to track user progress.

In [7]:
import re, html
FILLERS = r"\b(um+|uh+|hmm+|huh+|erm+|like)\b"
def normalize_input(s: str) -> str:
    if not s: return s
    s2 = re.sub(FILLERS, "", s, flags=re.IGNORECASE)
    s2 = re.sub(r"\s{2,}", " ", s2).strip()
    s2 = re.sub(r"\.{2,}", ".", s2)
    s2 = s2.strip(" \t\n\r-–—:;")
    return s2 if s2 else s

def extract_text(resp):
    try:
        txt = getattr(resp, "text", None)
        if txt and isinstance(txt, str) and txt.strip():
            return txt.strip()
    except Exception:
        pass
    try:
        if hasattr(resp, "to_dict"):
            d = resp.to_dict()
        elif isinstance(resp, dict):
            d = resp
        else:
            return str(resp)
        for key in ("content","contents","output","text"):
            if key in d:
                v = d[key]
                if isinstance(v, str) and v.strip():
                    return v.strip()
                if isinstance(v, list):
                    parts = []
                    for part in v:
                        if isinstance(part, dict) and "text" in part:
                            parts.append(part["text"])
                        elif isinstance(part, str):
                            parts.append(part)
                    if parts:
                        return "\n".join(parts)[:4000]
    except Exception:
        try:
            return str(resp)
        except:
            return "<unreadable response>"
    try:
        return str(resp)
    except:
        return "<no textual content>"

print("Response helpers ready.")

Response helpers ready.


**Cell 7 - Input Cleaning & Response Extraction**

This cell ensures user messages are clean and model responses are extracted safely.

Main functions

Removes fillers (“umm”, “like”, “hmm”) for cleaner conversations.

Extracts clean text from complex Gemini responses.

Handles edge cases where responses include lists, dicts, and nested nodes.

Why it matters

Reduces model confusion and prevents UI glitches.

In [8]:
emotion_system = """
You are EmotionAgent for StepCom: produce one empathetic validation sentence, provide the required emotional support,
one grounding exercise (<=3 steps), and an offer: 'Would you like a tiny plan?'
Keep it short and kind.
"""
plan_system = """
You are PlanAgent for StepCom: produce a 1-sentence micro-goal and 2-3 time-bound micro-steps (<=10 min each), plus a check-in phrase.
Keep it concise and action-focused.
"""

try:
    from google.generativeai import types as _types
    has_http_retry = hasattr(_types, "HttpRetryOptions")
except Exception:
    has_http_retry = False

if has_http_retry and retry_config is not None:
    EmotionAgent = genai.GenerativeModel(model_name=MODEL_NAME, system_instruction=emotion_system, http_retry=retry_config)
    PlanAgent = genai.GenerativeModel(model_name=MODEL_NAME, system_instruction=plan_system, http_retry=retry_config)
else:
    EmotionAgent = genai.GenerativeModel(model_name=MODEL_NAME, system_instruction=emotion_system)
    PlanAgent = genai.GenerativeModel(model_name=MODEL_NAME, system_instruction=plan_system)

print("EmotionAgent and PlanAgent instantiated using model:", MODEL_NAME)

def call_emotion_agent(user_text: str):
    prompt = f"User says: '''{user_text}'''\nRespond per your instructions."
    try:
        chat = EmotionAgent.start_chat(history=[])
        if not has_http_retry and 'send_with_retries' in globals():
            resp = send_with_retries(chat, prompt)
        else:
            resp = chat.send_message(prompt)
        text = extract_text(resp)
    except Exception as e:
        text = f"[EmotionAgent error: {type(e).__name__}]"
        traceback.print_exc()
    log_a2a_call("EmotionAgent", prompt, text)
    return text

def call_plan_agent(task_text: str):
    prompt = f"User task/goal: '''{task_text}'''\nReturn micro-plan."
    try:
        chat = PlanAgent.start_chat(history=[])
        if not has_http_retry and 'send_with_retries' in globals():
            resp = send_with_retries(chat, prompt)
        else:
            resp = chat.send_message(prompt)
        text = extract_text(resp)
    except Exception as e:
        text = f"[PlanAgent error: {type(e).__name__}]"
        traceback.print_exc()
    log_a2a_call("PlanAgent", prompt, text)
    return text

def controller_delegate_demo(user_input: str):
    # returns None or A2A combined response
    try:
        u = normalize_input(user_input or "")
        l = u.lower()
        emotional = any(k in l for k in ["i'm sad","i'm struggling","i can't","i cant","depressed","anxious","lonely","overwhelmed","i'm overwhelmed","i'm stressed","i'm stressed out"])
        planning  = any(k in l for k in ["task","finish","project","study","assignment","exam","homework","plan","start","deadline","due"])
        # if emotional -> EmotionAgent; optionally PlanAgent if it looks like planning too
        if emotional:
            emo = call_emotion_agent(u)
            if planning:
                plan = call_plan_agent(u)
                return emo + "\n\nPlan:\n" + plan
            return emo
        return None
    except Exception as e:
        traceback.print_exc()
        return "[A2A delegation error: couldn't process request]"

print("A2A delegation ready.")

EmotionAgent and PlanAgent instantiated using model: models/gemini-2.5-flash
A2A delegation ready.


**Cell 8 - EmotionAgent & PlanAgent**

Two specialized micro-agents are defined here:

EmotionAgent

Gives warmth, empathy, grounding exercises

Helps users who feel stressed, overwhelmed, anxious

PlanAgent

Produces tiny 2–3 step micro-plans

Focused on completing small tasks quickly (Gen Z style)

Controller Delegate Logic

Automatically triggers:

EmotionAgent when message contains emotional distress

PlanAgent when message contains task-related triggers

Both when message mixes emotion + task

Why this matters

This makes Stepcom feel like a real companion, not just a chatbot, but a system that understands the emotional + productivity context. 

In [9]:
MASTER_PROMPT = """
You are Stepcom — the user's AI productivity companion and friend.
Tone: Gen-Z friendly, gentle but real. Provide one tiny action <=10 minutes.
When user shares habits or task-like sentences, you may ask to save them internally.
When user reports completion, mark it. When asked, explain pros/cons for habits.
Do not create UI elements — operate through chat only. Keep responses short, warm, and actionable.
"""

tools = [add_task, complete_task, add_goal, add_habit, mark_habit_done, get_habits]

if has_http_retry and retry_config is not None:
    controller_model = genai.GenerativeModel(model_name=MODEL_NAME, system_instruction=MASTER_PROMPT, tools=tools, http_retry=retry_config)
else:
    controller_model = genai.GenerativeModel(model_name=MODEL_NAME, system_instruction=MASTER_PROMPT, tools=tools)

controller_chat = controller_model.start_chat(history=[])
print("Controller created with model:", MODEL_NAME)

Controller created with model: models/gemini-2.5-flash


**Cell 9 - Main Controller Model**

This is the brain of StepCom.

What it does

Integrates all tools + system prompts.

Enforces Stepcom’s personality:

Gen-Z friendly

Warm but real

Helpful accountability

Always gives a tiny actionable step

Processes all chat messages not handled by A2A agents.

Why it matters

This is the core intelligence layer responsible for keeping messages consistent, helpful, and on brand.

In [10]:
def explain_habit_with_gemini(name: str):
    if name in explain_cache:
        return explain_cache[name]
    h = memory.get("habits", {}).get(name)
    if not h:
        return "Habit not found."
    kind = h["type"]
    prompt = f"""
You are a compassionate Gen-Z friend. For the habit named '{name}' (type: {kind}), produce:
1) Two concise PROS (if good) OR two concise CONS (if bad).
2) Two short reasons/examples (<=1 sentence each).
3) One tiny 3-step micro-action the user can do today to reinforce/reduce it.
Keep tone friendly, validating, and actionable. Max 6 lines.
"""
    try:
        if has_http_retry and retry_config is not None:
            explain_model = genai.GenerativeModel(model_name=MODEL_NAME, http_retry=retry_config)
        else:
            explain_model = genai.GenerativeModel(model_name=MODEL_NAME)
        chat = explain_model.start_chat(history=[])
        if not has_http_retry and 'send_with_retries' in globals():
            resp = send_with_retries(chat, prompt)
        else:
            resp = chat.send_message(prompt)
        text = extract_text(resp)
    except Exception as e:
        text = f"Error generating habit explanation: {type(e).__name__}: {e}"
    explain_cache[name] = text
    save_json(EXPLAIN_CACHE_PATH, explain_cache)
    return text

print("explain_habit_with_gemini ready.")

explain_habit_with_gemini ready.


**Cell 10 - Habit Explanation Engine**

This cell builds the module that:

Generates

Pros of good habits

Cons of bad habits

Real examples

A 3-step micro action

Features

Uses caching to avoid repeating API calls

Integrates into Stepcom’s chat flow naturally

Why it matters

It transforms Stepcom into a true improvement coach - not only tracking habits, but explaining and motivating change.

In [11]:
import random

def motivation_of_day():
    items = [
        "Start tiny. Future-you will be thankful.",
        "5 minutes today > 0 minutes. Start now.",
        "Consistency > intensity. One small step."
    ]
    return random.choice(items)

def support_response(user_text=""):
    validations = [
        "I hear you — that feels heavy and valid.",
        "You're allowed to feel this; I'm with you.",
        "This is tough, and that doesn't make you weak."
    ]
    micro = [
        "1) 3 deep breaths\n2) Drink water\n3) Do one 5-min step",
        "1) Name one feeling\n2) Stretch 60s\n3) Try 5 minutes of focused work"
    ]
    return f"{random.choice(validations)}\n\nTry this:\n{random.choice(micro)}"

def reality_check(user_text=""):
    punches = [
        "Confidence is cute, action makes it real.",
        "Main character energy needs main character effort.",
        "You're hyped — now channel it into one measurable thing."
    ]
    push = [
        "Do ONE thing for 5 minutes now.",
        "Open a doc and write one paragraph.",
        "Start a 5-minute timer and begin. No perfection."
    ]
    return f"{random.choice(punches)} 💜\n\n{random.choice(push)}"

print("Personality helpers ready.")

Personality helpers ready.


**Cell 11 - Personality Helpers**

Short, reusable helpers for tone and motivation.

Includes

Motivation-of-the-day

Emotional support responses

Reality checks

Micro encouragement messages

Why it matters

These give Stepcom emotional depth and variety - key to keeping Gen Z users engaged.

In [12]:
HELP = """
StepCom Console Commands (for local debug only; the Gradio cell is the primary interface):
- add_habit <name> <good|bad>
- mark_habit_done <name>
- add_task <text>
- complete_task <text>
"""
print("Console helper included (commented). Use chat UI (Gradio) for interactive demo.")

Console helper included (commented). Use chat UI (Gradio) for interactive demo.


**Cell 12 - (Optional) Console Mode**

This is a non-UI testing interface.

Why it's here

Useful for debugging

Helps judges verify agent logic without UI

Left commented out so it doesn't block Kaggle runs

In [13]:
import gradio as gr
from datetime import datetime

CSS = r"""
/* Page + container */
body { background: linear-gradient(180deg,#06040c 0%, #0b0710 100%); color: #111111; font-family: Inter, system-ui, -apple-system, 'Segoe UI', Roboto, 'Helvetica Neue', Arial; }
.gradio-container { max-width:980px !important; margin: 18px auto; border-radius: 12px; padding: 14px; }

/* Title & subtitle */
.title { font-size:3rem; font-weight:600; color:#7248C2; font-family: 'Inter', sans-serif; letter-spacing: -0.02em }             /* darker purple title */
.subtitle { color:#9068DE; margin-top:4px; font-size:13px; }           /* DARKER subtitle */

/* Panel */
.section { background: #0f0816; padding:12px; border-radius:12px; margin-bottom:12px; box-shadow: 0 12px 30px rgba(0,0,0,0.6); }
.panel-title { color:#D6CBFF; font-weight:700; margin-bottom:8px; font-size:14px; }

/* Buttons */
.btn-primary {
  background: #3B1F6B !important;   /* dark purple (Send) */
  color: #FFFFFF !important;        /* white text */
  border:1.5px solid #000 !important;
  border-radius:10px !important;
  padding:10px 16px !important;
  font-weight:700;
  box-shadow: 0 6px 18px rgba(59,31,107,0.25);
}
.btn-primary:hover { background:#4b2a8a !important; transform: translateY(-1px); }

/* Make Micro-step and Clear darker and readable (previously ghost) */
.btn-ghost {
  background: #3B1F6B !important;   /* now dark purple like primary (but lighter shade optional) */
  color:#FFFFFF !important;         /* white text for readability */
  border:1.2px solid rgba(0,0,0,0.8) !important;
  border-radius:10px !important;
  padding:8px 12px !important;
  font-weight:700;
}
.btn-ghost:hover { background:#4b2a8a !important; }

/* Inputs & chat bubbles */
.input-compact .input, .input-compact textarea { background: #FFFFFF !important; color: #000 !important; border-radius:8px; border: 1px solid #111 !important; }
.chat-bubble-user { background:#FFFFFF; color:#000; padding:10px 14px; border-radius:14px; display:inline-block; margin:6px 0; }
.chat-bubble-assistant { background: linear-gradient(90deg,#6F4AE2,#B99CFF); color:#fff; padding:10px 14px; border-radius:14px; display:inline-block; margin:6px 0; box-shadow: 0 10px 24px rgba(122,86,246,0.12); }

/* Footer / small muted text (darker now) */
.small-muted { color:#9068DE; opacity:0.95; font-size:13px; }

/* Placeholder text slightly darker for readability */
.input-compact ::placeholder { color: #ACA2C0 !important; opacity:1 !important; }

/* Ensure footer text area visible */
footer { color: #9068DE !important; }
"""

with gr.Blocks(css=CSS, title="Stepcom - Your AI Productivity Companion") as app:
    # Header / movement title
    gr.HTML("""
    <div style='display:flex;align-items:center;justify-content:space-between'>
      <div>
        <div class='title'>Stepcom</div>
        <div class='subtitle'>Your AI Productivity Companion - chat to get micro-plans, reality checks & habit help</div>
      </div>
    </div>
    """)
    gr.Markdown("")  # spacer

    # Chat column only (wide)
    with gr.Group(elem_classes="section"):
        chat = gr.Chatbot(type="messages", label="")
        user_input = gr.Textbox(placeholder="Say something (e.g. 'I'm procrastinating', 'I keep scrolling', 'I can't start my assignment') - press send", show_label=False, elem_classes="input-compact")
        with gr.Row():
            send_btn = gr.Button("Send", elem_classes="btn-primary")
            micro_btn = gr.Button("Micro-step", elem_classes="btn-ghost")
            clear_btn = gr.Button("Clear", elem_classes="btn-ghost")
        gr.HTML("<div class='small-muted'>StepCom stores simple history & habit notes so it can follow up. All via chat.</div>")

    # Callback logic (unchanged)
    def send_message(user_text, history):
        history = history or []
        if not user_text or not user_text.strip():
            return history
        history.append({"role":"user","content":user_text})

        # A2A delegation (emotion/planning suggestions)
        a2a = controller_delegate_demo(user_text)
        if a2a:
            resp = a2a
        else:
            try:
                chat_obj = controller_chat.start_chat(history=[]) if hasattr(controller_chat, "start_chat") else controller_chat
                prompt = user_text + f"\n\n[SESSION_MODE:{memory.get('meta',{}).get('mode','soft')}]"
                if 'send_with_retries' in globals() and not USE_HTTP_RETRY:
                    r = send_with_retries(chat_obj, prompt)
                else:
                    r = chat_obj.send_message(prompt)
                resp = extract_text(r)
            except Exception as e:
                print("Controller call failed:", e)
                resp = support_response(user_text) if any(k in user_text.lower() for k in ["sad","struggl","can't","cant","lonely","stressed"]) else "I couldn't reach my brain right now — try rephrasing or say 'motivate me'."

        memory.setdefault("meta", {}).setdefault("a2a_logs", []).append({"ts": datetime.now().isoformat(), "prompt": user_text[:500], "response": str(resp)[:1000]})
        save_json(MEMORY_PATH, memory)
        history.append({"role":"assistant","content":resp})
        return history

    send_btn.click(send_message, [user_input, chat], chat)
    user_input.submit(send_message, [user_input, chat], chat)

    def micro_cb(history):
        history = history or []
        history.append({"role":"assistant","content":"Tiny step: set a 10-minute timer and do one focused thing. Say 'done' when you're back — I'll celebrate."})
        return history
    micro_btn.click(micro_cb, [chat], chat)

    clear_btn.click(lambda: [], None, chat)

# Launch UI (will print shareable URL)
app.launch(server_name="0.0.0.0", server_port=7860, share=True)

* Running on local URL:  http://0.0.0.0:7860
* Running on public URL: https://1ca9aa90ad50b49813.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


**Cell 13 - Gradio Chat UI (Primary User Experience)**

This cell builds the entire Stepcom interface.

Design goals

Dark purple gradient background

White chat bubbles

Black button borders

Clean, modern Gen Z aesthetic

Chat-only interface (no task boxes, no habit forms)

Features

Real-time chat

Micro-step quick button

Clear history button

Emotion + micro-plan agent routing

Memory auto-saving

A shareable public URL

In [14]:
OUT_DIR = "/kaggle/working"
MANIFEST_PATH = os.path.join(OUT_DIR, "stepcom_manifest.json")
TESTS_PATH = os.path.join(OUT_DIR, "stepcom_selftests.json")
WRITEUP_PATH = os.path.join(OUT_DIR, "writeup_template.md")

def now(): return datetime.now().isoformat()

manifest = {
    "name": "StepCom",
    "type": "Concierge / Chat-first Productivity Companion",
    "description": "Chat-first Gen-Z friendly AI that offers micro-plans, motivational support, habit explanations, and reality checks.",
    "model_used": MODEL_NAME,
    "features": {
        "multi_agent": ["Controller","EmotionAgent","PlanAgent"],
        "tools": ["add_task","complete_task","add_goal","add_habit","mark_habit_done","get_habits","explain_habit_with_gemini"],
        "memory_path": MEMORY_PATH,
        "a2a_logs": "memory.meta.a2a_logs",
        "retry": "HttpRetryOptions if available else fallback send_with_retries",
    },
    "created_at": now()
}
with open(MANIFEST_PATH, "w") as f:
    json.dump(manifest, f, indent=2)

# simple tests (not exhaustive)
tests = {}
errors = []
try:
    t0 = f"t-{int(time.time())}"
    r = add_task("test "+t0)
    tests['add_task'] = {'input': t0, 'result': r}
except Exception as e:
    tests['add_task'] = {'error': str(e)}
    errors.append(str(e))

try:
    hname = f"testhabit_{int(time.time())}"
    r1 = add_habit(hname, 'bad')
    r2 = mark_habit_done(hname)
    try:
        explain_text = explain_habit_with_gemini(hname)
    except Exception as e:
        explain_text = f"ERR: {e}"
    tests['habit_flow'] = {'add': r1, 'done': r2, 'explain_preview': (explain_text or '')[:300]}
except Exception as e:
    tests['habit_flow'] = {'error': str(e)}
    errors.append(str(e))

with open(TESTS_PATH, 'w') as f:
    json.dump({'tests': tests, 'errors': errors, 'generated_at': now()}, f, indent=2)

writeup_md = f"""
# Stepcom — Capstone Agent (Concierge/Chat-first)

Short pitch:
Stepcom is a chat-first, Gen-Z oriented productivity companion that focuses on tiny, actionable steps rather than heavy task management. The UI is a single chat playground — laziness-friendly by design.

Architecture: Controller delegates to EmotionAgent & PlanAgent for empathy and micro-plans. Tools manage tasks/habits/goals in persisted memory.

Run instructions: add your GOOGLE_API_KEY in Kaggle Secrets. Run all cells. Launch the Gradio UI (cell 13) and use the chat to interact.

"""
with open(WRITEUP_PATH, 'w') as f:
    f.write(writeup_md)

print("Manifest, tests, and writeup template written to /kaggle/working/.")


Manifest, tests, and writeup template written to /kaggle/working/.


**Cell 14 - Packaging (Manifest, Self-tests & Writeup Template)**

This cell produces all files needed for Kaggle Capstone submission:

Self-tests check

Adding tasks

Creating habits

Marking habits done

Habit explanation

A2A agent pipeline

Why this matters

Kaggle requires:

A manifest

Tests

A write-up

This cell builds all necessary deliverables automatically.